In [13]:
import yaml
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, HTML



def MakePsLink(y_title, y_link_prefix, y_id):
    return f'''<a target="_blank" href="{y_link_prefix + y_id}">{y_title}</a>"'''

METADATA_YAMLFILE = 'ps_md.yaml'
DB_YAMLFILE = 'ps_db.yaml'

db = yaml.safe_load(open(DB_YAMLFILE))
md = yaml.safe_load(open(METADATA_YAMLFILE))

site_mapping = { info['site_name']: info['link_prefix'] for info in md}

db_df = pd.DataFrame(db) #[['site', 'id', 'title', 'rank']]

record_counter = lambda x: len(x) if x is not None else 0
get_max_date = lambda x: max([record['date'] for record in x]) if x is not None else None
def get_max_solve_time(x):
    ret = [record['time'] for record in x if record is not None]
    ret = [t for t in ret if t is not None]
    if x is not None:
        return min(ret)
    else:
        return None
def get_min_solve_time(x):
    ret = [record['time'] for record in x if record is not None]
    ret = [t for t in ret if t is not None]
    if x is not None:
        return min(ret)
    else:
        return None
get_min_date = lambda x: min([record['date'] for record in x]) if x is not None else None
#get_max_solve_time = lambda x: max([record['time'] for record in x]) if x is not None else None
#get_min_solve_time = lambda x: min([record['time'] for record in x]) if x is not None else None

make_hyperlink = lambda row: f'<a href="{site_mapping.get(row["site"], "")}{row["id"]}">{row["site"] + " " + str(row["id"]) + " " + row["title"] + " " + row["rank"] }</a>'
db_df['unique_title'] = db_df.apply(make_hyperlink, axis=1)

db_df['tag_list'] = db_df['tags'].apply(lambda tags: '<ul>' + ''.join([f'<li>{tag}</li>' for tag in tags]) + '</ul>' if tags else '')


today = datetime.today().date()

db_df['date_first'] = db_df['records'].apply(get_min_date)
db_df['date_end'] = db_df['records'].apply(get_max_date)
db_df['date_range'] = db_df.apply(lambda row: str(int((today - row['date_end']).days)) if row['date_end'] else None, axis=1)
db_df['record_count'] = db_df['records'].apply(record_counter)
db_df['max_solve_time'] = db_df['records'].apply(get_max_solve_time)
db_df['min_solve_time'] = db_df['records'].apply(get_min_solve_time)

db_df['score'] = db_df['records'].apply(lambda records: sum(record['state'] for record in records if record['state'] == True) / len(records) * 100 if records else 0)
db_df['score'] = db_df['score'].apply(lambda x: round(x, 0))

display_db = db_df[['unique_title', 'tag_list', 'record_count', 'date_first', 'date_end', 'date_range', 'max_solve_time', 'min_solve_time', 'score']]
display_db = display_db.sort_values(by=['date_end'], ascending=False)

heading_properties = [('font-size', '5px')]
cell_properties = [('font-size', '5px')]
db_df_style = [dict(selector="th", props=heading_properties), dict(selector="td", props=cell_properties)]
db_df.style.set_table_styles(db_df_style)


display(HTML(display_db.to_html(escape=False)))


,unique_title,tag_list,record_count,date_first,date_end,date_range,max_solve_time,min_solve_time,score
31,백준 2579 계단 오르기 S3,Dynamic Programming,3,2022-08-22,2024-01-26,10,60,60,0.0
30,백준 23882 알고리즘 수업-선택 정렬2 B1,SortSelection Sort,1,2024-01-26,2024-01-26,10,19,19,100.0
29,백준 23881 알고리즘 수업 - 선택 정렬1 B1,SortSelection Sort,1,2024-01-26,2024-01-26,10,60,60,100.0
17,백준 2206 벽 부수고 이동하기 G3,State BFS,4,2023-12-20,2024-01-24,12,86,86,25.0
28,백준 16930 달리기 P3,BFS,1,2024-01-24,2024-01-24,12,120,120,0.0
26,백준 1303 전쟁 - 전투 S1,BFSConnected Components,1,2024-01-23,2024-01-23,13,33,33,100.0
25,백준 6603 로또 S2,CombinationPermutationBrute-Force AlgorithmDFS,1,2024-01-23,2024-01-23,13,33,33,0.0
24,백준 1987 알파벳 G4,BFS,1,2024-01-23,2024-01-23,13,70,70,0.0
19,백준 1525 퍼즐 G2,State BFS,3,2024-01-19,2024-01-22,14,50,50,33.0
18,백준 1600 말이 되고픈 원숭이 G3,State BFS,2,2024-01-18,2024-01-22,14,60,60,100.0
